# SVM

## Video

<iframe width="100%" height="500"
src="https://www.youtube.com/embed/_YPScrckx28" 
frameborder="0" 
allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" 
allowfullscreen></iframe>


## 

https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html

Which is licenced under CC-by licence.

## Animation


<iframe width="100%" height="500"
src="https://www.youtube.com/embed/9NrALgHFwTo" 
frameborder="0" 
allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" 
allowfullscreen></iframe>

<iframe width="100%" height="500"
src="https://www.youtube.com/embed/3liCbRZPrZA" 
frameborder="0" 
allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" 
allowfullscreen></iframe>


### Hinge loss

An alternative to cross-entropy for binary 
classification problems is the hinge loss function.
It is primarily developed for use with 
Support Vector Machine (SVM) models.

It is intended for use with binary 
classification where the target values are in the set {-1, 1}.


## Kernel trick

<iframe width="100%" height="500"
src="https://www.youtube.com/embed/Q7vT0--5VII" 
frameborder="0" 
allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" 
allowfullscreen></iframe>


## Support Vector Regression (SVR)




## SVM v.s. logistic regression



## SVM v.s. linear regression


## Acknowledgement